# Enkel Billedklassificering

Denne notebook viser dig, hvordan du klassificerer billeder ved hjælp af et forudtrænet neuralt netværk.

**Hvad du vil lære:**
- Hvordan man indlæser og bruger en forudtrænet model
- Forbehandling af billeder
- At lave forudsigelser på billeder
- Forståelse af tillidsscorer

**Anvendelse:** Identificer objekter i billeder (som "kat", "hund", "bil" osv.)

---


## Trin 1: Importér nødvendige biblioteker

Lad os importere de værktøjer, vi har brug for. Bare rolig, hvis du ikke forstår dem alle endnu!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Trin 2: Indlæs forudtrænet model

Vi vil bruge **MobileNetV2**, et neuralt netværk, der allerede er trænet på millioner af billeder.

Dette kaldes **Transfer Learning** - at bruge en model, som andre har trænet!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Trin 3: Hjælpefunktioner

Lad os oprette funktioner til at indlæse og forberede billeder til vores model.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Trin 4: Test på eksempler fra billeder

Lad os prøve at klassificere nogle billeder fra internettet!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klassificer Hvert Billede


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Trin 5: Prøv dine egne billeder!

Erstat URL'en nedenfor med en hvilken som helst billed-URL, du ønsker at klassificere.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Hvad skete der lige?

1. **Vi indlæste en forudtrænet model** - MobileNetV2 blev trænet på millioner af billeder
2. **Vi forbehandlede billeder** - Ændrede størrelse og formaterede dem til modellen
3. **Modellen lavede forudsigelser** - Den gav sandsynligheder for 1000 objektklasser
4. **Vi dekodede resultaterne** - Konverterede tal til menneskelæselige etiketter

### Forståelse af tillidsscores

- **90-100%**: Meget sikker (næsten helt korrekt)
- **70-90%**: Sikker (sandsynligvis korrekt)
- **50-70%**: Noget sikker (kan være korrekt)
- **Under 50%**: Ikke særlig sikker (usikker)

### Hvorfor kan forudsigelser være forkerte?

- **Usædvanlig vinkel eller belysning** - Modellen blev trænet på typiske fotos
- **Flere objekter** - Modellen forventer ét hovedobjekt
- **Sjældne objekter** - Modellen kender kun 1000 kategorier
- **Lav kvalitet billede** - Slørede eller pixelerede billeder er sværere at analysere

---


## 🚀 Næste skridt

1. **Prøv forskellige billeder:**
   - Find billeder på [Unsplash](https://unsplash.com)
   - Højreklik → "Kopiér billedadresse" for at få URL'en

2. **Eksperimentér:**
   - Hvad sker der med abstrakt kunst?
   - Kan det genkende objekter fra forskellige vinkler?
   - Hvordan håndterer det flere objekter?

3. **Lær mere:**
   - Udforsk [Computer Vision-lektioner](../lessons/4-ComputerVision/README.md)
   - Lær at træne din egen billedklassifikator
   - Forstå hvordan CNN'er (Convolutional Neural Networks) fungerer

---

## 🎉 Tillykke!

Du har lige bygget en billedklassifikator ved hjælp af et avanceret neuralt netværk!

Denne samme teknik driver:
- Google Photos (organisering af dine billeder)
- Selvkørende biler (genkendelse af objekter)
- Medicinsk diagnose (analyse af røntgenbilleder)
- Kvalitetskontrol (detektering af fejl)

Bliv ved med at udforske og lære! 🚀



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal det bemærkes, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
